<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/AutoGPT_with_LangChain_Primitives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#!pip install langchain openai tiktoken google-search-results chromadb -qU

In [1]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

C:\Users\I541988\AppData\Local\Temp\ipykernel_16196\3646097473.py:5: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  import llm_commons.proxy.base


In [2]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [3]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

os.environ['AICORE_LLM_RESOURCE_GROUP'] = 'default'
os.environ["TAVILY_API_KEY"] = sk['tavily']

os.environ["BING_SUBSCRIPTION_KEY"] = sk['bing']
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['LLM_COMMONS_PROXY'] = 'aicore'

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [4]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()
embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)
llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')

In [5]:
from langchain_community.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = BingSearchAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]


In [6]:
from langchain.vectorstores import Chroma

vectordb = Chroma(persist_directory="./.chroma", embedding_function=embedding)

In [7]:
from langchain_experimental.autonomous_agents import AutoGPT
from llm_commons.langchain.proxy import ChatOpenAI

In [8]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Xiaoming",
    ai_role="Assistant",
    tools=tools,
    llm=llm,
    memory=vectordb.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True


In [9]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
  agent.run(["最好的中考数学辅导书是什么？"])
  print(cb)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Xiaoming, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 最好的中考数学辅导书是什么？


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"t

In [25]:
!cat weather_report.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.
